In [4]:
import os
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
#from utils import helpers
from organelle_heatmap import get_mask, get_average_intensities_cr, get_average_intensities_tsp, correlation
from organelle_heatmap_helpers import get_cells, plot_pilr_collage, plot_image_collage, get_heatmap

import configs.config as cfg
intensity_sampling_concentric_ring = False #True
intensity_warping = True
cell_line = 'all_celllines'#'S-BIAD34' #'U2OS' #'Hep-G2' #'U2OS' # 

project_dir = os.path.join(os.path.dirname(cfg.PROJECT_DIR), cell_line)
print('Working dir: ', project_dir)

fft_dir = f"{project_dir}/fftcoefs/{cfg.ALIGNMENT}"
fft_path = os.path.join(fft_dir, f"fftcoefs_{cfg.N_COEFS}.txt")
shape_mode_path = f"{project_dir}/shapemode/{cfg.ALIGNMENT}_{cfg.MODE}"
if intensity_sampling_concentric_ring:
    avg_organelle_dir = f"{project_dir}/matrix_protein_avg"
    sampled_intensity_dir = f"{project_dir}/sampled_intensity_bin"
if intensity_warping:
    avg_organelle_dir = f"{project_dir}/warps_protein_avg_cell_pairwise" 
    sampled_intensity_dir = f"{project_dir}/warps" 
    
id_keep, mask = get_mask(file_path=f"{shape_mode_path}/Avg_cell.npz")
df_trans = pd.read_csv(f"{project_dir}/shapemode/{cfg.ALIGNMENT}_{cfg.MODE}/transformed_matrix.csv")

Working dir:  /data/2Dshapespace/all_celllines


In [5]:

df_trans#.groupby('GMM_cc_label').agg({'pseudotime':'mean', 'cell_id':'count'})

,Unnamed: 0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC1015,PC1016,PC1017,PC1018,PC1019,PC1020,PC1021,PC1022,PC1023,PC1024
0,0,10080.834647,685.258868,-4597.448614,2052.931009,60.662864,-32.714465,-1155.732199,210.515731,628.517353,...,0,0,0,0,0,0,0,0,0,0
1,1,2222.373366,-13118.298427,-11596.993211,1916.255434,-10420.590426,-9044.797360,-3475.433975,-1888.890900,-2316.090463,...,0,0,0,0,0,0,0,0,0,0
2,2,17599.805187,-1249.290906,-6277.301413,8040.331066,-680.339962,-3336.304790,4954.500262,-193.062844,-2080.541480,...,0,0,0,0,0,0,0,0,0,0
3,3,-3236.041666,1019.483196,-11867.886420,-5481.344911,2954.731049,-5822.081647,1302.187904,525.741228,-1591.176683,...,0,0,0,0,0,0,0,0,0,0
4,4,1217.732056,-39080.480972,2821.761605,-15373.119916,-6407.154104,-729.550636,5653.268168,-448.895644,3310.833533,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85175,85175,-20513.061811,7784.153814,3152.843808,-169.358044,-955.846067,-569.652491,966.225171,-649.663306,-1767.567754,...,0,0,0,0,0,0,0,0,0,0
85176,85176,-18184.903699,9636.697043,-994.255174,1292.061437,-372.478377,-306.402694,1863.553462,-464.627336,-4416.961374,...,0,0,0,0,0,0,0,0,0,0
85177,85177,-17359.456142,10063.457858,3689.942605,951.279234,110.792179,-419.887942,2177.634442,-1124.951740,-3321.366802,...,0,0,0,0,0,0,0,0,0,0
85178,85178,-23499.446976,5474.496097,-2151.772701,-9.613132,-397.290415,106.847852,523.319651,262.405042,-454.822230,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
# X_train, X_test, y_train, y_test = train_test_split(df[['nu_area','cell_area']], df['GMM_cc'], test_size=0.1, random_state=42, stratify=df['GMM_cc'])

tmp = sc_stats[(sc_stats.GMM_cc_label == 'G2') ].dropna() # & (~sc_stats.PC1.isna())]
X_train, X_test, y_train, y_test = train_test_split(tmp[['nu_area', 'cell_area']], #tmp[[f'PC{n}' for n in range(1,11,1)]], #
                                                    tmp['pseudotime_bin'], 
                                                    test_size=0.40, random_state=99, stratify=tmp['pseudotime_bin'])


#clf = LinearDiscriminantAnalysis()
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
cm = confusion_matrix(y_test, pred)
labels = set(y_test)
# Plot heatmap
plt.figure(figsize=(6, 5))
print(f'x_train {X_train.shape[0]}, x_test {y_test.shape[0]}, Accuracy {acc}, \n {y_test.value_counts()}')
sns.heatmap(cm, annot=True,  xticklabels=labels, yticklabels=labels)